In [1]:
import os
import sys
import datetime
import math
import pandas as pd
import numpy as np
import quant_utils.data_moudle as dm
from quant_utils.db_conn import DB_CONN_JJTG_DATA
file_path = "C:/Users/Wilcoxon/Desktop/配置excel/"


写入基金投顾产品池监控报表

In [2]:

file_name = "基金投顾产品池监控报表.xlsx"
df1 = pd.read_excel(file_path+file_name, sheet_name= '基金投顾产品池产品状态监控')
df2 = pd.read_excel(file_path+file_name, sheet_name= '基金投顾产品池费用分成数据')
df1["产品代码"] = df1["产品代码"].apply(lambda s: str(s).rjust(6, "0"))
df2["产品代码"] = df2["产品代码"].apply(lambda s: str(s).rjust(6, "0"))
df = df1.merge(df2, how='left')
columns = [
    "产品代码", "产品风险等级", 
    "是否基金投顾备选池", "产品池类型",
    "个人首次申购最低金额", "管理费分成_个人", "管理费分成_机构",
    "管理费率_公开", "销售服务费分成比例", "销售服务费_公开", 
    "延迟交付天数", "ta编号",
    "个人单笔最高申购金额","机构单笔最高申购金额", "个人单日最高认申购金额",
    "机构单日最高认申购金额", "单笔赎回最低份额", "首次入池时间",
]
df = df[columns]
df = df.drop_duplicates()
rename_columns = {
    "产品代码" : "TICKER_SYMBOL", 
    "产品风险等级": "RISK_LEVEL", 
    "是否基金投顾备选池": "IF_IN_TRANCHE", 
    "产品池类型" : "TRANCHE",
    "个人首次申购最低金额": "FIRST_BUY", 
    "管理费分成_个人": "MANAGEMENT_FEE_INDIVIDUAL",
    "管理费分成_机构": "MANAGEMENT_FEE_INSTITUTION",
    "管理费率_公开": "MANAGEMENT_FEE", 
    "销售服务费分成比例": "SALE_FEE_PROPORTION", 
    "销售服务费_公开": "SALE_FEE", 
    "延迟交付天数": "DELAY_DATE", 
    "ta编号": "TA",
    "个人单笔最高申购金额": "MAX_BUY_INDIVIDUAL",
    "机构单笔最高申购金额": "MAX_BUY_INSTITUTION", 
    "个人单日最高认申购金额": "MAX_BUY_DAILY_INDIVIDUAL",
    "机构单日最高认申购金额": "MAX_BUY__DAILY_INSTITUTION", 
    "单笔赎回最低份额": "MIN_REDEEM",
    "首次入池时间": "FIRST_IN_TRANCHE_DATE"
}
df.rename(columns=rename_columns, inplace=True)
map_dict = {
    "是":1, "否": 0, "未入库": 2
}
df.loc[df["MAX_BUY_DAILY_INDIVIDUAL"]==0, "MAX_BUY_DAILY_INDIVIDUAL"] = 99999999999.99
df.loc[df["MAX_BUY__DAILY_INSTITUTION"]==0, "MAX_BUY__DAILY_INSTITUTION"] = 99999999999.99
df["IF_IN_TRANCHE"] = df["IF_IN_TRANCHE"].map(map_dict)
df.loc[df["TRANCHE"]=='基础池',"IF_IN_TRANCHE"] = 0
DB_CONN_JJTG_DATA.upsert(df, table="portfolio_basic_products")

d:\miniforge3\envs\my312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\miniforge3\envs\my312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


TIME:2024-12-20 10:43:45,717 - Level:INFO - Mesaage:portfolio_basic_products更新插入9404行数据


upsert耗时:3.5212 s


更新兴业证券BOP产品

In [3]:

for i in os.listdir(file_path):
    if i.startswith("代销产品清单") and i.endswith(".xlsx"):
        bop = pd.read_excel(file_path+i, engine="openpyxl")
        bop = bop[["基金代码", "TA编号", "TA名称", "基金名称", "风险等级"]]
        bop.rename(
            columns={
                "基金代码": "TICKER_SYMBOL",
                "TA编号": "TA",
                "TA名称": "TA_NAME",
                "基金名称": "FUND_NAME",
                "风险等级": "RISK_LEVEL",
            },
            inplace=True
        )
        DB_CONN_JJTG_DATA.upsert(bop, table="XY_BOP")

d:\miniforge3\envs\my312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


TIME:2024-12-20 10:43:48,853 - Level:INFO - Mesaage:XY_BOP更新插入12347行数据


upsert耗时:1.5579 s


写入资配系统净值

In [4]:
result_df = []
counter = 1
for i in os.listdir(file_path+"组合净值/"):
    print(counter, i)
    counter += 1
    df = pd.read_excel(file_path+"组合净值/" + i, header=1)
    df.rename(
        columns={
            "日期": 'TRADE_DT',
            "投资组合(扣除投顾费,申赎费)": "PORTFOLIO_NAV",
            "业绩基准": "BENCHMARK_NAV"
        },
        inplace=True
    )
    df["PORTFOLIO_NAME"] = i[0: i.index("-数")]
    # result_df.append(df)
    # result_df = pd.concat(result_df)
    DB_CONN_JJTG_DATA.upsert(df, table="portfolio_nav")

TIME:2024-12-20 10:43:48,974 - Level:INFO - Mesaage:portfolio_nav更新插入426行数据


1 兴证全明星精选-蚂蚁-数据视图-20241220.xlsx

upsert耗时:0.0820 s
2 兴证小确幸精选-蚂蚁-数据视图-20241220.xlsx



TIME:2024-12-20 10:43:49,066 - Level:INFO - Mesaage:portfolio_nav更新插入426行数据
TIME:2024-12-20 10:43:49,154 - Level:INFO - Mesaage:portfolio_nav更新插入426行数据
TIME:2024-12-20 10:43:49,214 - Level:INFO - Mesaage:portfolio_nav更新插入262行数据
TIME:2024-12-20 10:43:49,278 - Level:INFO - Mesaage:portfolio_nav更新插入262行数据


upsert耗时:0.0690 s
3 兴证月月享精选-蚂蚁-数据视图-20241220.xlsx

upsert耗时:0.0630 s
4 知己优选-1000增强-数据视图-20241220.xlsx

upsert耗时:0.0430 s
5 知己优选-300增强-数据视图-20241220.xlsx



TIME:2024-12-20 10:43:49,360 - Level:INFO - Mesaage:portfolio_nav更新插入262行数据
TIME:2024-12-20 10:43:49,407 - Level:INFO - Mesaage:portfolio_nav更新插入178行数据


...

upsert耗时:0.2420 s
10 知己优选-海外权益-数据视图-20241220.xlsx

upsert耗时:0.0299 s
11 知己优选-现金宝-数据视图-20241220.xlsx

upsert耗时:0.0650 s
12 知己优选-短债增强-数据视图-20241220.xlsx



TIME:2024-12-20 10:43:50,310 - Level:INFO - Mesaage:portfolio_nav更新插入1087行数据


upsert耗时:0.0390 s
13 知己优选-货币小福星-数据视图-20241220.xlsx

upsert耗时:0.1310 s
14 知己优选-进取全明星-数据视图-20241220.xlsx



TIME:2024-12-20 10:43:50,709 - Level:INFO - Mesaage:portfolio_nav更新插入2181行数据
TIME:2024-12-20 10:43:50,751 - Level:INFO - Mesaage:portfolio_nav更新插入107行数据
TIME:2024-12-20 10:43:50,798 - Level:INFO - Mesaage:portfolio_nav更新插入128行数据
TIME:2024-12-20 10:43:50,870 - Level:INFO - Mesaage:portfolio_nav更新插入265行数据


upsert耗时:0.3101 s
15 知己私享-全球资产配置-数据视图-20241220.xlsx

upsert耗时:0.0300 s
16 知己私享-安稳宝-数据视图-20241220.xlsx

upsert耗时:0.0330 s
17 知己私享-稳确幸-数据视图-20241220.xlsx

upsert耗时:0.0540 s


fof基金分类

In [5]:
query_sql = """
SELECT
	a.TICKER_SYMBOL,
	a.IDX_SYMBOL,
	b.SEC_TAG,
	b.SEC_SHORT_NAME,
	a.WEIGHT * 100 as EQUITY_PORT
FROM
	fund_benchmark a
	JOIN index_tag b ON a.IDX_SYMBOL = b.TICKER_SYMBOL 
WHERE
	1 = 1 
	AND a.IS_EXE = 1 
	AND b.FIRST_TAG = '资产类别' 
	AND `LEVEL` = 2 
	AND a.TICKER_SYMBOL IN ( 
        SELECT TICKER_SYMBOL 
        FROM `chentiancheng`.`fund_type_jy` 
        WHERE `INDUSTRIESNAME_1` = 'FOF' AND CUR_SIGN = 1 
    ) 
"""
df = DB_CONN_JJTG_DATA.exec_query(query_sql)
df.loc[(df["SEC_TAG"]=='基金') & (df["SEC_SHORT_NAME"].str.contains("股")), "SEC_TAG"] = '股票'
df = df[df["SEC_TAG"] == '股票'][["TICKER_SYMBOL", "EQUITY_PORT"]]
df = df.groupby(by=["TICKER_SYMBOL"]).sum().reset_index()
df["INNER_TYPE"] = None
df.loc[(df["EQUITY_PORT"]>=5)&((df["EQUITY_PORT"]<15)), "INNER_TYPE"] = '知己优选-月月享'
df.loc[(df["EQUITY_PORT"]>=15)&((df["EQUITY_PORT"]<25)), "INNER_TYPE"] = '知己优选-固收佳进取'
df.loc[(df["EQUITY_PORT"]>=45)&((df["EQUITY_PORT"]<55)), "INNER_TYPE"] = '知己优选-均衡小确幸'
df.loc[(df["EQUITY_PORT"]>=75)&((df["EQUITY_PORT"]<85)), "INNER_TYPE"] = '知己私享-稳确幸'
df.loc[(df["EQUITY_PORT"]>=90), "INNER_TYPE"] = '知己优选-进取全明星'
df["INNER_TYPE"] = df["INNER_TYPE"].fillna("其他")
DB_CONN_JJTG_DATA.upsert(df, table="fof_type")

TIME:2024-12-20 10:43:51,488 - Level:INFO - Mesaage:fof_type更新插入905行数据


exec_query耗时:0.5103 s

upsert耗时:0.0890 s
